# NEXT_seq Worm genome Alignment


In this notebook, 
-   import reference genome and its annotations
-   trim fasta files from the Worm Study
-   

## Import Genome

In [7]:
!mkdir genome 
    #creates the directory that will hold the reference genome and it's annotated gene files
!wget -nc -O genome/genome.fa.gz https://ftp.ebi.ac.uk/pub/databases/wormbase/parasite/releases/WBPS19/species/schistosoma_mansoni/PRJEA36577/schistosoma_mansoni.PRJEA36577.WBPS19.genomic.fa.gz 
    #downloads the file from the given url, in this case it is downloading the genome fasta file the genetic sequence for schistosoma_mansoni
    #-O allows the specification of the output directory
    #-nc prevents overwritting a file that already exists
!gzip -d -f genome/genome.fa.gz
    # the gzip takes the sequence data from the wget and compresses it so that the file can be transferred easier and saves storage space

mkdir: cannot create directory ‘genome’: File exists
--2024-12-19 13:49:59--  https://ftp.ebi.ac.uk/pub/databases/wormbase/parasite/releases/WBPS19/species/schistosoma_mansoni/PRJEA36577/schistosoma_mansoni.PRJEA36577.WBPS19.genomic.fa.gz
Resolving ftp.ebi.ac.uk... 193.62.193.165
Connecting to ftp.ebi.ac.uk|193.62.193.165|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 116797085 (111M) [application/x-gzip]
Saving to: ‘genome/genome.fa.gz’

genome/genome.fa.gz 100%[===================>] 111.39M   588KB/s    in 3m 14s  

2024-12-19 13:53:13 (587 KB/s) - ‘genome/genome.fa.gz’ saved [116797085/116797085]



Import the annotations for known genes in worm genomes

In [2]:
!wget -O - https://ftp.ebi.ac.uk/pub/databases/wormbase/parasite/releases/WBPS19/species/schistosoma_mansoni/PRJEA36577/schistosoma_mansoni.PRJEA36577.WBPS19.canonical_geneset.gtf.gz | \
        #grabs the annotation file from the given url and outputs it to the specified directory
    gzip -f -d > genome/annotations.gtf
        #-f is to force it to overwrite any existing annotation file and -d decompress the file so it's not as large

--2024-11-21 16:30:04--  https://ftp.ebi.ac.uk/pub/databases/wormbase/parasite/releases/WBPS19/species/schistosoma_mansoni/PRJEA36577/schistosoma_mansoni.PRJEA36577.WBPS19.canonical_geneset.gtf.gz
Resolving ftp.ebi.ac.uk... 193.62.193.165
Connecting to ftp.ebi.ac.uk|193.62.193.165|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3228084 (3.1M) [application/x-gzip]
Saving to: ‘STDOUT’

-                   100%[===================>]   3.08M  3.82MB/s    in 0.8s    

2024-11-21 16:30:06 (3.82 MB/s) - written to stdout [3228084/3228084]



#### Index the genome for efficient gene referencing

In [8]:
!samtools faidx genome/genome.fa
    #creates an index file that allows for efficient random access to the sequences in the fasta file

## CutAdapt Trimming the Genome
    In this step we prepare the reads to have a higher quality to give the read a higher probility of success in aligning

Each of the following cutadapt calls takes the samples and trims them based on the specified instructions
- -j sets cutadapt to use, 0 automatically detects the number available
- -m sets the minimum length of a read to 20, that way any reads that are too small to process won't cause problems in the alignment steps
- --nextseq-trim accounts for the instruments used for sequencing that encode G, the normal quality trimmer is unable to handle this problem so this is used rather than q=10
- -u and -U removes the first 10 base pairs from the start of both of-  the paired end reads, this was chosed because qc output indicated that these bp were causing data to be less optimal
- -a and -A with adapter sequence, removes the adapter used by next seq 
        removing the adapter improves alignment since the adapter is not supposed to be in the gene of a worm so it will most likely not be able to find alignments if we don't remove the adapter that isn't part of the genes in a worm
- after specifying how we want to cut the reads, the location of the trimmed reads is given as -o for the forward end and -p for the reverse read
- then the location of the files that we are trimming is specified namining the forward read and then the reverse read

In [1]:
!mkdir fastq_trimmed_sub
    #creates the directory that will hold the trimmed fastq files 

!cutadapt -j 0 -m 20 --nextseq-trim=10 -u 10 -U 10 -a AGATCGGAAGAGCACACGTCTGAACTCCAGTCA -A AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT -o ./fastq_trimmed_sub/Int_01_1.fastq.gz -p ./fastq_trimmed_sub/Int_01_2.fastq.gz /data/classes/2024/fall/biol343/course_files/20240923_LH00283_0144_B22NMVKLT3/subsample/Int-01_S42_L005_R1_001.fastq.gz /data/classes/2024/fall/biol343/course_files/20240923_LH00283_0144_B22NMVKLT3/subsample/Int-01_S42_L005_R2_001.fastq.gz
!cutadapt -j 0 -m 20 --nextseq-trim=10 -u 10 -U 10 -a AGATCGGAAGAGCACACGTCTGAACTCCAGTCA -A AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT -o ./fastq_trimmed_sub/Int_02_1.fastq.gz -p ./fastq_trimmed_sub/Int_02_2.fastq.gz /data/classes/2024/fall/biol343/course_files/20240923_LH00283_0144_B22NMVKLT3/subsample/Int-02_S43_L005_R1_001.fastq.gz /data/classes/2024/fall/biol343/course_files/20240923_LH00283_0144_B22NMVKLT3/subsample/Int-02_S43_L005_R2_001.fastq.gz
!cutadapt -j 0 -m 20 --nextseq-trim=10 -u 10 -U 10 -a AGATCGGAAGAGCACACGTCTGAACTCCAGTCA -A AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT -o ./fastq_trimmed_sub/Int_03_1.fastq.gz -p ./fastq_trimmed_sub/Int_03_2.fastq.gz /data/classes/2024/fall/biol343/course_files/20240923_LH00283_0144_B22NMVKLT3/subsample/Int-03_S44_L005_R1_001.fastq.gz /data/classes/2024/fall/biol343/course_files/20240923_LH00283_0144_B22NMVKLT3/subsample/Int-03_S44_L005_R2_001.fastq.gz
!cutadapt -j 0 -m 20 --nextseq-trim=10 -u 10 -U 10 -a AGATCGGAAGAGCACACGTCTGAACTCCAGTCA -A AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT -o ./fastq_trimmed_sub/Int_04_1.fastq.gz -p ./fastq_trimmed_sub/Int_04_2.fastq.gz /data/classes/2024/fall/biol343/course_files/20240923_LH00283_0144_B22NMVKLT3/subsample/Int-04_S45_L005_R1_001.fastq.gz /data/classes/2024/fall/biol343/course_files/20240923_LH00283_0144_B22NMVKLT3/subsample/Int-04_S45_L005_R2_001.fastq.gz
!cutadapt -j 0 -m 20 --nextseq-trim=10 -u 10 -U 10 -a AGATCGGAAGAGCACACGTCTGAACTCCAGTCA -A AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT -o ./fastq_trimmed_sub/Liv_01_1.fastq.gz -p ./fastq_trimmed_sub/Liv_01_2.fastq.gz /data/classes/2024/fall/biol343/course_files/20240923_LH00283_0144_B22NMVKLT3/subsample/Liv-01_S38_L005_R1_001.fastq.gz /data/classes/2024/fall/biol343/course_files/20240923_LH00283_0144_B22NMVKLT3/subsample/Liv-01_S38_L005_R2_001.fastq.gz
!cutadapt -j 0 -m 20 --nextseq-trim=10 -u 10 -U 10 -a AGATCGGAAGAGCACACGTCTGAACTCCAGTCA -A AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT -o ./fastq_trimmed_sub/Liv_02_1.fastq.gz -p ./fastq_trimmed_sub/Liv_02_2.fastq.gz /data/classes/2024/fall/biol343/course_files/20240923_LH00283_0144_B22NMVKLT3/subsample/Liv-02_S39_L005_R1_001.fastq.gz /data/classes/2024/fall/biol343/course_files/20240923_LH00283_0144_B22NMVKLT3/subsample/Liv-02_S39_L005_R2_001.fastq.gz
!cutadapt -j 0 -m 20 --nextseq-trim=10 -u 10 -U 10 -a AGATCGGAAGAGCACACGTCTGAACTCCAGTCA -A AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT -o ./fastq_trimmed_sub/Liv_03_1.fastq.gz -p ./fastq_trimmed_sub/Liv_03_2.fastq.gz /data/classes/2024/fall/biol343/course_files/20240923_LH00283_0144_B22NMVKLT3/subsample/Liv-03_S40_L005_R1_001.fastq.gz /data/classes/2024/fall/biol343/course_files/20240923_LH00283_0144_B22NMVKLT3/subsample/Liv-03_S40_L005_R2_001.fastq.gz
!cutadapt -j 0 -m 20 --nextseq-trim=10 -u 10 -U 10 -a AGATCGGAAGAGCACACGTCTGAACTCCAGTCA -A AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT -o ./fastq_trimmed_sub/Liv_04_1.fastq.gz -p ./fastq_trimmed_sub/Liv_04_2.fastq.gz /data/classes/2024/fall/biol343/course_files/20240923_LH00283_0144_B22NMVKLT3/subsample/Liv-04_S41_L005_R1_001.fastq.gz /data/classes/2024/fall/biol343/course_files/20240923_LH00283_0144_B22NMVKLT3/subsample/Liv-04_S41_L005_R2_001.fastq.gz

This is cutadapt 4.9 with Python 3.12.3
Command line parameters: -j 32 -m 20 --nextseq-trim=10 -u 10 -U 10 -a AGATCGGAAGAGCACACGTCTGAACTCCAGTCA -A AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT -o ./fastq_trimmed_sub/Int_01_1.fastq.gz -p ./fastq_trimmed_sub/Int_01_2.fastq.gz /data/classes/2024/fall/biol343/course_files/20240923_LH00283_0144_B22NMVKLT3/subsample/Int-01_S42_L005_R1_001.fastq.gz /data/classes/2024/fall/biol343/course_files/20240923_LH00283_0144_B22NMVKLT3/subsample/Int-01_S42_L005_R2_001.fastq.gz
Processing paired-end reads on 32 cores ...
Done           00:00:20    12,421,987 reads @   1.6 µs/read;  36.58 M reads/minute
Finished in 20.452 s (1.646 µs/read; 36.44 M reads/minute).

=== Summary ===

Total read pairs processed:         12,421,987
  Read 1 with adapter:               3,346,061 (26.9%)
  Read 2 with adapter:               3,263,630 (26.3%)

== Read fate breakdown ==
Pairs that were too short:             253,185 (2.0%)
Pairs written (passing filters):    12,168,802 (98.0%)

In [2]:
!mkdir fastq_trimmed_qc_sub
!fastqc -t 16 fastq_trimmed_sub/*.fastq.gz -o fastq_trimmed_qc_sub 

application/gzip
application/gzip
Started analysis of Int_01_1.fastq.gz
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
Started analysis of Int_01_2.fastq.gz
Started analysis of Int_02_1.fastq.gz
Started analysis of Int_02_2.fastq.gz
Started analysis of Int_03_1.fastq.gz
Started analysis of Int_03_2.fastq.gz
Started analysis of Int_04_1.fastq.gz
Started analysis of Int_04_2.fastq.gz
Started analysis of Liv_01_1.fastq.gz
Started analysis of Liv_01_2.fastq.gz
Approx 5% complete for Int_01_1.fastq.gz
Started analysis of Liv_02_1.fastq.gz
Started analysis of Liv_02_2.fastq.gz
Approx 5% complete for Int_01_2.fastq.gz
Started analysis of Liv_03_1.fastq.gz
Started analysis of Liv_03_2.fastq.gz
Started analysis of Liv_04_1.fastq.gz
Started analysis of Liv_04_2.fastq.gz
Approx 5% complete for Int_03_1.fastq.

In [4]:
!mkdir fastq_qc_sub
!fastqc /data/classes/2024/fall/biol343/course_files/20240923_LH00283_0144_B22NMVKLT3/subsample/*fastq.gz -o fastq_qc_sub

application/gzip
application/gzip
Started analysis of Int-01_S42_L005_R1_001.fastq.gz
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
Approx 5% complete for Int-01_S42_L005_R1_001.fastq.gz
Approx 10% complete for Int-01_S42_L005_R1_001.fastq.gz
Approx 15% complete for Int-01_S42_L005_R1_001.fastq.gz
Approx 20% complete for Int-01_S42_L005_R1_001.fastq.gz
Approx 25% complete for Int-01_S42_L005_R1_001.fastq.gz
Approx 30% complete for Int-01_S42_L005_R1_001.fastq.gz
Approx 35% complete for Int-01_S42_L005_R1_001.fastq.gz
Approx 40% complete for Int-01_S42_L005_R1_001.fastq.gz
Approx 45% complete for Int-01_S42_L005_R1_001.fastq.gz
Approx 50% complete for Int-01_S42_L005_R1_001.fastq.gz
Approx 55% complete for Int-01_S42_L005_R1_001.fastq.gz
Approx 60% complete for Int-01_S42_L005_R1_001.fastq.gz
Appro

In [5]:
!multiqc -f -d fastq_qc_sub/ fastq_trimmed_qc_sub/ -n multiqc_report_all_ipynb_sub.html


  /// ]8;id=152343;https://multiqc.info\MultiQC]8;;\ 🔍 | v1.17

|           multiqc | Prepending directory to sample names
|           multiqc | Search path : /data/users/mccallke0364/Final_Project/Sm_Mira_IvT/subfiles/fastq_qc_sub
|           multiqc | Search path : /data/users/mccallke0364/Final_Project/Sm_Mira_IvT/subfiles/fastq_trimmed_qc_sub
|         searching | ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 64/64  _trimmed_qc_sub/Liv_04_2_fastqc.html
|            fastqc | Found 32 reports
|           multiqc | Report      : multiqc_report_all_ipynb_sub.html   (overwritten)
|           multiqc | Data        : multiqc_report_all_ipynb_sub_data   (overwritten)
|           multiqc | MultiQC complete


In [6]:
!mkdir genome/star
!mkdir alignment_sub_2
!mkdir alignment_sub_2/star

In [19]:
!STAR \
--runThreadN 16  \
--runMode genomeGenerate \
--genomeDir genome/star \
--genomeFastaFiles genome/genome.fa \
--sjdbGTFfile genome/annotations.gtf \
--sjdbOverhang 139\
--genomeSAindexNbases 13

	/data/users/mccallke0364/.conda/envs/final/bin/STAR-avx2 --runThreadN 16 --runMode genomeGenerate --genomeDir genome/star --genomeFastaFiles genome/genome.fa --sjdbGTFfile genome/annotations.gtf --sjdbOverhang 139 --genomeSAindexNbases 13
	STAR version: 2.7.10b   compiled: 2023-05-25T06:56:23+0000 :/opt/conda/conda-bld/star_1684997536154/work/source
Dec 15 17:25:42 ..... started STAR run
Dec 15 17:25:42 ... starting to generate Genome files
Dec 15 17:25:48 ..... processing annotations GTF
Dec 15 17:25:50 ... starting to sort Suffix Array. This may take a long time...
Dec 15 17:25:52 ... sorting Suffix Array chunks and saving them to disk...
Dec 15 17:26:50 ... loading chunks from disk, packing SA...
Dec 15 17:27:00 ... finished generating suffix array
Dec 15 17:27:00 ... generating Suffix Array index
Dec 15 17:27:37 ... completed Suffix Array index
Dec 15 17:27:37 ..... inserting junctions into the genome indices
Dec 15 17:28:13 ... writing Genome to disk ...
Dec 15 17:28:14 ... writi

In [10]:
!STAR \
--runThreadN 16\
--runMode alignReads \
--genomeDir genome/star \
--readFilesManifest manifest.tsv \
--readFilesCommand zcat \
--outSAMtype BAM SortedByCoordinate \
--outSAMunmapped Within \
--outFileNamePrefix alignment_sub_2/star/

!cp alignment_sub_2/star/Log.final.out alignment_sub_2/star/firstlog.final.out

	/data/users/mccallke0364/.conda/envs/final/bin/STAR-avx2 --runThreadN 16 --runMode alignReads --genomeDir genome/star --readFilesManifest manifest.tsv --readFilesCommand zcat --outSAMtype BAM SortedByCoordinate --outSAMunmapped Within --outFileNamePrefix alignment_sub_2/star/
	STAR version: 2.7.10b   compiled: 2023-05-25T06:56:23+0000 :/opt/conda/conda-bld/star_1684997536154/work/source
Dec 18 19:32:45 ..... started STAR run
Dec 18 19:32:45 ..... loading genome
Dec 18 19:32:48 ..... started mapping
Dec 18 23:58:55 ..... finished mapping
Dec 18 23:58:58 ..... started sorting BAM
Dec 19 00:05:27 ..... finished successfully


In [11]:
!STAR\
--runThreadN 16 \
--runMode alignReads \
--genomeDir genome/star \
--readFilesManifest manifest.tsv \
--readFilesCommand zcat \
--outSAMtype BAM SortedByCoordinate \
--outSAMunmapped Within \
--outSAMattributes NH HI AS nM RG \
--outFileNamePrefix alignment_sub_2/star/ \
--sjdbFileChrStartEnd alignment_sub_2/star/SJ.out.tab

!cp alignment_sub_2/star/Log.final.out alignment_sub_2/star/second.final.out

	/data/users/mccallke0364/.conda/envs/final/bin/STAR-avx2 --runThreadN 16 --runMode alignReads --genomeDir genome/star --readFilesManifest manifest.tsv --readFilesCommand zcat --outSAMtype BAM SortedByCoordinate --outSAMunmapped Within --outSAMattributes NH HI AS nM RG --outFileNamePrefix alignment_sub_2/star/ --sjdbFileChrStartEnd alignment_sub_2/star/SJ.out.tab
	STAR version: 2.7.10b   compiled: 2023-05-25T06:56:23+0000 :/opt/conda/conda-bld/star_1684997536154/work/source
Dec 19 00:05:33 ..... started STAR run
Dec 19 00:05:36 ..... loading genome
Dec 19 00:05:45 ..... inserting junctions into the genome indices
Dec 19 00:06:08 ..... started mapping
Dec 19 04:46:45 ..... finished mapping
Dec 19 04:46:48 ..... started sorting BAM
Dec 19 04:53:33 ..... finished successfully


In [12]:
!samtools flagstat alignment_sub_2/star/Aligned.sortedByCoord.out.bam

252673365 + 0 in total (QC-passed reads + QC-failed reads)
3664337 + 0 secondary
0 + 0 supplementary
0 + 0 duplicates
15397899 + 0 mapped (6.09% : N/A)
249009028 + 0 paired in sequencing
124504514 + 0 read1
124504514 + 0 read2
11733514 + 0 properly paired (4.71% : N/A)
11733514 + 0 with itself and mate mapped
48 + 0 singletons (0.00% : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)


In [13]:
!qualimap bamqc -nt 32 -outdir qualimap_2/star/bam -bam alignment_sub_2/star/Aligned.sortedByCoord.out.bam --feature-file genome/annotations.gtf 
!qualimap rnaseq -outdir qualimap_2/star/rnaseq -bam alignment_sub_2/star/Aligned.sortedByCoord.out.bam -gtf genome/annotations.gtf

Java memory size is set to 1200M
Launching application...

QualiMap v.2.3
Built on 2023-05-19 16:57

Selected tool: bamqc
Available memory (Mb): 35
Max memory (Mb): 1258
Starting bam qc....
Loading sam header...
Loading locator...
Loading reference...
Number of windows: 400, effective number of windows: 409
Chunk of reads size: 1000
Number of threads: 32
Initializing regions from genome/annotations.gtf.....
Found 241537 regions
Filling region references... 
Processed 50 out of 409 windows...
Processed 100 out of 409 windows...
Processed 150 out of 409 windows...
Processed 200 out of 409 windows...
Processed 250 out of 409 windows...
Processed 300 out of 409 windows...
Processed 350 out of 409 windows...
Processed 400 out of 409 windows...
Total processed windows:409
Number of reads: 249009028
Number of valid reads: 11733562
Number of correct strand reads:0

Inside of regions...
Num mapped reads: 9718104
Num mapped first of pair: 4863165
Num mapped second of pair: 4854939
Num singletons

In [14]:
!multiqc --force -d fastq_qc_sub/ fastq_trimmed_qc_sub/ alignment_sub_2/ -n post_align.html


  /// ]8;id=471038;https://multiqc.info\MultiQC]8;;\ 🔍 | v1.17

|           multiqc | Prepending directory to sample names
|           multiqc | Search path : /data/users/mccallke0364/Final_Project/Sm_Mira_IvT/subfiles/fastq_qc_sub
|           multiqc | Search path : /data/users/mccallke0364/Final_Project/Sm_Mira_IvT/subfiles/fastq_trimmed_qc_sub
|           multiqc | Search path : /data/users/mccallke0364/Final_Project/Sm_Mira_IvT/subfiles/alignment_sub_2
|           multiqc | Search path : /data/users/mccallke0364/Final_Project/Sm_Mira_IvT/subfiles
|         searching | ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 373/373  0malimap/star/rnaseq/css/agogo.css
|          qualimap | Found 2 BamQC reports
|          qualimap | Found 2 RNASeq reports
|              star | Found 3 reports
|            fastqc | Found 32 reports
|           multiqc | Report      : multiqc_report.html   (overwritten)
|           multiqc | Data        : multiqc_data   (overwritten)
|           multiqc | M

In [19]:
!mkdir counting_sub_2
!mkdir counting_sub_2/logs_sub
!mkdir counting_sub_2/dedup_sub

In [9]:
# !samtools view -H alignment_sub/star/Aligned.sortedByCoord.out.bam

INFO	2024-12-18 19:28:28	AddOrReplaceReadGroups	

********** NOTE: Picard's command line syntax is changing.
**********
********** For more information, please see:
********** 
https://github.com/broadinstitute/picard/wiki/Command-Line-Syntax-Transition-For-Users-(Pre-Transition)
**********
********** The command line looks like this in the new syntax:
**********
**********    AddOrReplaceReadGroups -I alignment_sub/star/Aligned.sortedByCoord.out.bam -O alignment_sub/star/Aligned_with_correct_RG.bam -RGID Int-01 -RGLB lib1 -RGPL illumina -RGPU unit1 -RGSM Int-01
**********


19:28:28.068 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/data/users/mccallke0364/.conda/envs/final/share/picard-3.2.0-0/picard.jar!/com/intel/gkl/native/libgkl_compression.so
[Wed Dec 18 19:28:28 CST 2024] AddOrReplaceReadGroups INPUT=alignment_sub/star/Aligned.sortedByCoord.out.bam OUTPUT=alignment_sub/star/Aligned_with_correct_RG.bam RGID=Int-01 RGLB=lib1 RGPL=illumina RGPU=unit1 RGSM

In [20]:
!picard MarkDuplicates -I alignment_sub_2/star/Aligned.sortedByCoord.out.bam -M counting_sub_2/logs_sub/star_duplicates -O counting_sub_2/dedup_sub/star.bam -Xmx100g --VALIDATION_STRINGENCY SILENT

09:27:42.544 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/data/users/mccallke0364/.conda/envs/final/share/picard-3.2.0-0/picard.jar!/com/intel/gkl/native/libgkl_compression.so
[Thu Dec 19 09:27:42 CST 2024] MarkDuplicates --INPUT alignment_sub_2/star/Aligned.sortedByCoord.out.bam --OUTPUT counting_sub_2/dedup_sub/star.bam --METRICS_FILE counting_sub_2/logs_sub/star_duplicates --VALIDATION_STRINGENCY SILENT --MAX_SEQUENCES_FOR_DISK_READ_ENDS_MAP 50000 --MAX_FILE_HANDLES_FOR_READ_ENDS_MAP 8000 --SORTING_COLLECTION_SIZE_RATIO 0.25 --TAG_DUPLICATE_SET_MEMBERS false --REMOVE_SEQUENCING_DUPLICATES false --TAGGING_POLICY DontTag --CLEAR_DT true --DUPLEX_UMI false --FLOW_MODE false --FLOW_DUP_STRATEGY FLOW_QUALITY_SUM_STRATEGY --USE_END_IN_UNPAIRED_READS false --USE_UNPAIRED_CLIPPED_END false --UNPAIRED_END_UNCERTAINTY 0 --UNPAIRED_START_UNCERTAINTY 0 --FLOW_SKIP_FIRST_N_FLOWS 0 --FLOW_Q_IS_KNOWN_END false --FLOW_EFFECTIVE_QUALITY_THRESHOLD 15 --ADD_PG_TAG_TO_READS 

In [23]:
!featureCounts -T 32 \
    counting_sub_2/dedup_sub/star.bam \
    -T 32 \
    -p \
    --byReadGroup \
    -s 1 \
    --ignoreDup \
    -M \
    --fraction \
    -a genome/annotations.gtf \
    -o star_counts_2.tsv \
    --verbose


        ==========     _____ _    _ ____  _____  ______          _____  
        =====         / ____| |  | |  _ \|  __ \|  ____|   /\   |  __ \ 
          =====      | (___ | |  | | |_) | |__) | |__     /  \  | |  | |
            ====      \___ \| |  | |  _ <|  _  /|  __|   / /\ \ | |  | |
              ====    ____) | |__| | |_) | | \ \| |____ / ____ \| |__| |
        ==========   |_____/ \____/|____/|_|  \_\______/_/    \_\_____/
	  v2.0.6

//========================== featureCounts setting ===========================\\
||                                                                            ||
||             Input files : 1 BAM file                                       ||
||                                                                            ||
||                           star.bam                                         ||
||                                                                            ||
||             Output file : star_counts_2.tsv                   

In [22]:
!multiqc --force -d fastq_qc_sub/ fastq_trimmed_qc_sub/ alignment_sub_2/ counting_sub_2/ -n alignment_qc.html


  /// ]8;id=74092;https://multiqc.info\MultiQC]8;;\ 🔍 | v1.17

|           multiqc | Prepending directory to sample names
|           multiqc | Search path : /data/users/mccallke0364/Final_Project/Sm_Mira_IvT/subfiles/fastq_qc_sub
|           multiqc | Search path : /data/users/mccallke0364/Final_Project/Sm_Mira_IvT/subfiles/fastq_trimmed_qc_sub
|           multiqc | Search path : /data/users/mccallke0364/Final_Project/Sm_Mira_IvT/subfiles/alignment_sub_2
|           multiqc | Search path : /data/users/mccallke0364/Final_Project/Sm_Mira_IvT/subfiles/counting_sub_2
|         searching | ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 75/75  qc_sub/Liv_03_1_fastqc.html.html
|            picard | Found 1 MarkDuplicates reports
|              star | Found 1 reports
|            fastqc | Found 32 reports
|           multiqc | Report      : alignment_qc.html
|           multiqc | Data        : alignment_qc_data
|           multiqc | MultiQC complete


In [14]:
!mkdir plots
!Rscript differential_expressions.R

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: S4Vectors
Loading required package: stats4
Loading required package: BiocGenerics

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:lubridate’:

    intersect, setdiff, union

The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    